# Problem Set #2
## Name: Sahithi Adari
### Date: 02/24/2021

In [1]:
import pandas as pd
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor # Calculating VIF
from sklearn.model_selection import cross_val_score, validation_curve, train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler

## Part 1

### Question 1

In [2]:
# The density function of X for an observeration that belongs to the 'yes' class
yes = (1 / (math.sqrt(2*math.pi*36))) * math.exp(-(1/2) * ((4-10)**2) /36)

In [3]:
# The density function of X for an observeration that belongs to the 'no' class
no = (1 / (math.sqrt(2*math.pi*36))) * math.exp(-(1/2) * ((4-0)**2) /36)

In [4]:
# Plugging in the above values into Bayes' Theorem to determine the overall probability
bayes = (0.8 * yes) / (0.8 * yes + 0.2 * no)

# Outputting the results
bayes

0.7518524532975261

The probability that a company will issue dividends this year given that its percentage profit was $X = 4$ last year is $75.19$%.

### Question 2

#### Part A

If there are no computational limitations then best subset selection will have the smallest training RSS. This is beceause, best subset selection iterates through $2^k$ models that involve subsets of *k* predictors.

#### Part B

Although a low training error doesn't guarantee a low test error, best subset selection is the most likely, once again, to give you the lowest test RSS given the large search space.

#### Part C

* True - Forward stepwise selection begins with a null model and add predictors to the model, one at a time, until all predictors are included and chooses the best one. Therefore, it naturally follows, that the *k*-variable model is a subset of the $(k + 1)$ model.
* True - Backwards stepwise selection begins with the full model and removes predictions to the model, one at a time, until it reaches the null model and chooses the best one. Therefore, much like the above question, it naturally follows that *k*-variable model is a subset of the $(k + 1)$ model as well.
* False - There is no link between backwards stepwise and forward stepwise.
* False - There is no link between forward stepwise and backwards stepwise.
* False - Best subset selection selects the best number of predictors for the *k*-variable model and the $(k+1)$-variable model respectively by running $2^k$ models. There is no overlap between the two models as the inclusion of an additional variable results in a search space that has doubled from the previous search space.

## Part 2

In [5]:
# Reading in the the CSV file
airbnb = pd.read_csv('nyc_airbnb_listings.csv')

In [6]:
# Viewing the data as a dataframe 
airbnb

,listing_id,host_id,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_group,...,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,2060,2259,0.22,0.50,0.0,0.0,0.0,1.0,0.0,Manhattan,...,0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.01
1,2595,2845,0.87,0.38,0.0,6.0,6.0,1.0,1.0,Manhattan,...,5,94.0,9.0,9.0,10.0,10.0,10.0,9.0,0,0.38
2,3831,4869,0.83,0.96,0.0,1.0,1.0,1.0,1.0,Brooklyn,...,69,90.0,9.0,9.0,10.0,10.0,10.0,8.0,0,4.71
3,5099,7322,NaN,0.71,0.0,1.0,1.0,1.0,0.0,Manhattan,...,8,90.0,10.0,9.0,10.0,10.0,10.0,9.0,0,0.59
4,5114,7345,0.50,NaN,0.0,3.0,3.0,1.0,0.0,Manhattan,...,0,94.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50791,42890680,258247777,1.00,1.00,0.0,5.0,5.0,1.0,0.0,Manhattan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
50792,42890730,79065824,1.00,0.99,0.0,21.0,21.0,1.0,1.0,Manhattan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
50793,42891018,11849741,NaN,NaN,0.0,1.0,1.0,1.0,0.0,Manhattan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
50794,42891637,209711461,NaN,1.00,0.0,2.0,2.0,1.0,0.0,Brooklyn,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


### Problem 1

#### Attribute Types

In [7]:
# Listing the datatypes of each attribute
airbnb.dtypes

listing_id                       int64
host_id                          int64
host_response_rate             float64
host_acceptance_rate           float64
host_is_superhost              float64
host_listings_count            float64
host_total_listings_count      float64
host_has_profile_pic           float64
host_identity_verified         float64
neighbourhood_group             object
room_type                       object
accommodates                     int64
bathrooms                      float64
bedrooms                       float64
beds                           float64
price                            int64
security_deposit               float64
cleaning_fee                   float64
guests_included                  int64
extra_people                     int64
has_availability                 int64
availability_30                  int64
availability_60                  int64
availability_90                  int64
availability_365                 int64
number_of_reviews        

Of what I can see of the dataset it appears that the majority of the attributes are a quantitative, continuous, ratio or interval. There are some obvious and notable exceptions to this by way of *listing_id*, *host_id* and *neighbourhood_group* (quantitative, discrete, nominal); *instant_bookable*, *host_is_superhost*, *host_has_profile_pic*, *host_identity_verified*(quantitative, discrete, binary); etc. It is also interesting to note that *neighbourhood_type* and *room_type* are objects which mean they likely have a non-numeric value.

#### Resolution and possible alternative units of analysis

The resolution is at an individual NYC Airbnb listing in March 2020. A secondary units of analysis could also be by host (each row representing the number of units in the NYC areas by host).

#### Dimensionality

In [8]:
# Counting the number of attributes (columns)
len(airbnb.columns)

36

In [9]:
# Counting the number of oberservations
len(airbnb)

50796

There are 36 attributes and 50796 observations.

#### Missing Values

In [10]:
# Determining the number of missing values per attribute
airbnb.isna().sum()

listing_id                         0
host_id                            0
host_response_rate             19006
host_acceptance_rate           14015
host_is_superhost                  5
host_listings_count                5
host_total_listings_count          5
host_has_profile_pic               5
host_identity_verified             5
neighbourhood_group                0
room_type                          0
accommodates                       0
bathrooms                         54
bedrooms                          77
beds                             482
price                              0
security_deposit               17325
cleaning_fee                   10528
guests_included                    0
extra_people                       0
has_availability                   0
availability_30                    0
availability_60                    0
availability_90                    0
availability_365                   0
number_of_reviews                  0
number_of_reviews_ltm              0
r

As the above code shows us there are a wide range of attributes with missing values with the majority of them clustering around the review score section. This makes sense as large and populous cities are more likely to have new, unreviewed Airbnb listings pop up on a daily basis. There is also the consideration to make regarding the timing of this data. Since this data was scraped in March 2020 (when COVID lockdown measures were just starting to take place in NYC) I wonder how many of these new, unreviewed listings were for people moving out of the city and trying to sublease their apartments.

#### Potential multicollinearity

In order to check for multicollinearity I decide to calculate the VIF of the data by utilizing the `variance_infation_package` from `statsmodels`.  This process did involve some amount of preprocessing and as such, I created a new dataframe.

In [11]:
# Creating a new dataframe for checking multicollinearity
MC = airbnb 

# Dropping the 'object' attributes from measurement 
MC = MC.drop(['neighbourhood_group', 'room_type'], axis = 1)

# Removing all NA values from measurement 
MC = MC.dropna()

In [12]:
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = MC.columns 
  
# Calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(MC.values, i) 
                          for i in range(len(MC.columns))] 
  
print(vif_data)

/Users/endor/Library/Python/3.8/lib/python/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


                        feature         VIF
0                    listing_id    2.733295
1                       host_id    1.980093
2            host_response_rate    1.183979
3          host_acceptance_rate    1.602412
4             host_is_superhost    1.268003
5           host_listings_count         inf
6     host_total_listings_count         inf
7          host_has_profile_pic    1.009585
8        host_identity_verified    1.370925
9                  accommodates    3.571461
10                    bathrooms    1.322428
11                     bedrooms    2.419052
12                         beds    2.837802
13                        price    1.136262
14             security_deposit    1.232824
15                 cleaning_fee    1.738215
16              guests_included    1.904379
17                 extra_people    1.119206
18             has_availability  791.659218
19              availability_30    7.634408
20              availability_60   33.579834
21              availability_90 

According to the above data, the section surrounding availability is highly multicollinear and this makes sense as properties that have availability for the next 30, 60, or 90 days, must have availability today/at the time of measurement. You cannot have *availability_30*, *availability_60* or *availability_90* without *has_availability*.

### Question 2

In [13]:
# Removing all NA values from the dataset and resetting the index 
airbnb = airbnb.dropna().reset_index(drop = True)

# Converting the categorical variables into dummy variables
airbnb = pd.get_dummies(airbnb)

In [14]:
# Splitting the data in the test and training
y = airbnb['price']
X = airbnb.drop('price', axis=1)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state = 10)

### Question 3

In [15]:
# Creating 'r2_scores' function that outputs the r^2 score as well as the adjusted r^2 scores
def r2_scores(y_true, y_pred, p):
    '''
    This function takes the true values, predicted values, as well as the number of predictors in order calucluate the
     r^2 score as well as the adjusted r^2 scores.
    
    Args:
        y_true (numeric, array): A numeric value or array of true y values
        y_pred (numeric, array): A numeric value or array of the predicted y values 
        p (numeric): number of predictors

    Returns:
        (r2, adjust_r2) (tuple): A tuple of the r^2 score and the adjusted r^2 scores
    '''
    RSS = sum((y_true - y_pred) ** 2)
    avg = sum(y_true)/len(y_true) # Mean
    TSS = sum((y_true - avg) ** 2)
    r2 = 1 - (RSS/TSS)
    adjust_r2 = 1 - ((RSS / (20114 - p))/(TSS/20114))
    return (r2, adjust_r2)

### Question 4

In [16]:
# Fitting the model with training data (linear regression)
lm = LinearRegression()
lin_reg = lm.fit(Xtrain, ytrain)

In [17]:
# Standardizing features to eliminate issues of scaling
scaler = StandardScaler()
XSC = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

In [18]:
#Resplitting the data given the values were scaled
XSCtrain, XSCtest, ysctrain, ysctest = train_test_split(XSC, y, test_size = 0.3, random_state = 10)

In [19]:
# Fitting the model with training data (LASSO)
# Alpha of 0.5
lasso = Lasso(alpha = 0.5)
lasso_half = lasso.fit(XSCtrain, ysctrain)

In [20]:
# Fitting the model with training data (LASSO)
# Alpha of 5.0
lasso = Lasso(alpha = 5.0)
lasso_5 = lasso.fit(XSCtrain, ysctrain)

If we look at the documentation for `sklearnit Lasso` function we can see that a default alpha value of 0 is the same as the linear regression model. This is because the alpha is a constant that multiplies the $l_1$ penalty. When we increase the alpha to 0.5, 5, 5.0 etc. we increase the penalty and, thereby, increasing the number of coefficients estimates that exactly equal to 0 provided that the tuning parameter is sufficiently large enough.

### Question 5

In [21]:
# Generating a prediction for the linear regression 
lin_reg_pred = lin_reg.predict(Xtest)

# Linear Regression r2_scores
r2_scores(ytest, lin_reg_pred, 36)

(0.18370016824862345, 0.18223653671445428)

In [22]:
# Generating a prediction for the LASSO with alpha of 0.5
lasso_half_pred = lasso_half.predict(XSCtest)

# LASSO with alpha of 0.5 r2_scores
r2_scores(ysctest, lasso_half_pred, 36)

(0.1861131623711305, 0.18465385735296935)

In [23]:
# Generating a prediction for the LASSO with alpha of 5
lasso_5_pred = lasso_5.predict(XSCtest)

# LASSO with alpha of 5 r2_scores
r2_scores(ysctest, lasso_5_pred, 36)

(0.19036096753979803, 0.1889092788671929)

$R^2$ scores per model:
* Linear Regression - 0.1837
* LASSO $\alpha$ of 0.5 - 0.1861
* LASSO $\alpha$ of 5 - 0.1904

$Adjusted R^2$ scores per model:
* Linear Regression - 0.1822
* LASSO $\alpha$ of 0.5 - 0.1847
* LASSO $\alpha$ of 5 - 0.1889

What these models are generally telling us is that our prediction model is doing a pretty terrible job regardless of the model being used. Given that the $R^2$ is near 0 across all 3 models this means that the regression did not explain much of the variability in the response. This is further supported by the $Adjusted R^2$ score since that measurement is looking for a large value.

As mentioned above, given that an $\alpha$ of 0 is the linear regression model, it make sense that there is minimal difference between the $R^2$ and $Adjusted R^2$ scores of the linear regression model and the LASSO with an $\alpha$ of 0.5.

Of the three models present the LASSO with an $\alpha$ of 5 presented the best results indicating that if we increase the $\alpha$ we're more likely to get a better prediction utilizing only the most important coefficients. On the flip side we risk overfitting the data as we increase the $\alpha$.

It is also important to note that since I did no preprocessing (i.e. removing outliers, rescaling attributes as needed, etc.) which could explain my really low $R^2$ and $Adjusted R^2$ scores.

### Question 6

In [24]:
# Creating a dataframe of the variables and the respective model's coefficients
pred_coef = pd.DataFrame({'Variable': X.columns, 'LM Coef': lin_reg.coef_, 'Lasso Coef': lasso_5.coef_})

In [25]:
# Sorting by LASSO (decending order)
pred_coef.sort_values('Lasso Coef', ascending = False, key=pd.Series.abs)

,Variable,LM Coef,Lasso Coef
9,accommodates,2.182019e+01,39.988453
35,neighbourhood_group_Manhattan,7.507681e+01,33.747584
29,review_scores_location,4.589311e+01,22.281335
11,bedrooms,3.334848e+01,20.721258
10,bathrooms,5.493134e+01,19.825965
14,cleaning_fee,3.089652e-01,19.582661
26,review_scores_cleanliness,3.567671e+01,18.098535
30,review_scores_value,-2.382794e+01,-15.407135
28,review_scores_communication,-1.705024e+01,-11.216012
1,host_id,1.210643e-07,10.043409


Starting with the LASSO we can see that, under that model, 18 different coefficients estimated were forced to be exactly zero. We can interpret this to mean that of all the predictor coefficients possible these 18 were likely to be the least predictive. If we sort the dataframe by descending order (absolute value) we can see that the top 5 most predictive coefficients, according to LASSO, are: *accomodates*, *neighbourhood_group_Manhattan*, *review_scores_location*, *bedrooms*, and *bathrooms*.

The Linear Regression model, on the other hand, calculates *neighbourhood_group_Manhattan*, *host_response_rate*, *room_type_Shared room*, *room_type_Hotel room*, and *bathroom* to be the most predictive of price. Every coefficient was considered in the Linear Regression model when building the prediction as there are no shrinkage penalties.

Across both models an Airbnb being located in Manhattan and having multiple bathrooms are the most predictive of price. And this make sense given that, of the 5 boroughs, Manhattan is the most expensive one and the one with the most amount of tourists.

There are also some interesting differences between the two models. For example the Linear Regression model ranks *host_has_profile_pic* highly (6th highest) whereas the LASSO shrinks that value to 0. On the other hand *cleaning_fee* is ranked highly by LASSO (6th highest once again) whereas the Linear Regression model does not.

When comparing the model we can also see how the LASSO reduces the magnitude of the coefficients. The linear model coefficient on *neighbourhood_group_Manhattan* is 75.08 whereas on the LASSO the coefficient is 33.75. This is more than likely due to the standardization of our values but is still important to note.